<div align="center"><h1 style="color:red">DS-Assignment Model Building</h1></div>

---

In [1]:
#!pip install lightfm
import lightfm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score,recall_at_k,reciprocal_rank
from copy import deepcopy
import os

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
final_train = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')

In [3]:
final_train.head()

,user_id,pratilipi_id,read_percent,updated_at,month,year,reading_time_min,genre|action,genre|adventure,genre|and,...,genre|spiritual,genre|story,genre|suspense,genre|swahindi2,genre|the,genre|translation,genre|webseries,genre|wellness,genre|women,train
0,5506791949051549,1377786227473306,100.0,2022-03-23 00:06:07.280,1.0,2022.0,4,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
1,5506791949051549,1377786227547613,100.0,2022-03-23 00:06:05.988,2.0,2022.0,5,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
2,5506791963035645,1377786228336911,100.0,2022-03-23 00:06:01.889,3.0,2022.0,11,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
3,5506791981255160,1377786222836248,100.0,2022-03-23 00:03:56.342,2.0,2021.0,3,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
4,5506791981255160,1377786218892660,100.0,2022-03-23 00:03:56.059,4.0,2020.0,4,0.432808,0.432808,0.423382,...,0.0,0.0,0.331406,0.0,0.0,0.0,0.0,0.0,0.0,True


In [4]:
final_test.head()

,user_id,pratilipi_id,read_percent,updated_at,month,year,reading_time_min,genre|action,genre|adventure,genre|and,...,genre|spiritual,genre|story,genre|suspense,genre|swahindi2,genre|the,genre|translation,genre|webseries,genre|wellness,genre|women,train
0,5506791992372558,1377786218111595,100.0,2022-03-23 00:08:25.250,2.0,2020.0,29,0.000000,0.000000,0.000000,...,0.0,0.0,0.681687,0.0,0.0,0.0,0.0,0.0,0.0,False
1,5506791959279525,1377786225901639,100.0,2022-03-23 00:08:16.603,10.0,2021.0,10,0.000000,0.000000,0.000000,...,0.0,0.0,0.478954,0.0,0.0,0.0,0.0,0.0,0.0,False
2,5506791973513254,1377786227564949,90.0,2022-03-23 00:08:14.542,2.0,2022.0,8,0.565112,0.565112,0.552805,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False
3,5506791954805098,1377786220151688,100.0,2022-03-23 00:08:10.493,7.0,2020.0,5,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False
4,5506791976871041,1377786225173250,15.0,2022-03-23 00:08:04.897,8.0,2021.0,12,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False


In [5]:
final_train.columns

Index(['user_id', 'pratilipi_id', 'read_percent', 'updated_at', 'month',
       'year', 'reading_time_min', 'genre|action', 'genre|adventure',
       'genre|and', 'genre|awards', 'genre|challenge', 'genre|chat',
       'genre|children', 'genre|comedy', 'genre|crime', 'genre|cyber',
       'genre|detective', 'genre|drama', 'genre|entertainment',
       'genre|erotica', 'genre|experiences', 'genre|family', 'genre|fantasy',
       'genre|fiction', 'genre|health', 'genre|hindi', 'genre|horror',
       'genre|indiawale', 'genre|inspiring', 'genre|jeevan', 'genre|kalamkar',
       'genre|kids', 'genre|krishi', 'genre|lekhan', 'genre|life',
       'genre|marathon', 'genre|memories', 'genre|moral', 'genre|murder',
       'genre|mystery', 'genre|mythology', 'genre|novels', 'genre|parytan',
       'genre|politics', 'genre|pratilipi', 'genre|pravasi',
       'genre|premkamahina', 'genre|radio', 'genre|rashtriya',
       'genre|relegion', 'genre|romance', 'genre|sahitya', 'genre|samman',
       'g

In [6]:
# Interaction data from complete data 
interactions_current = final_train[['user_id', 'pratilipi_id', 'read_percent',"updated_at"]]
interactions_new = final_test[['user_id', 'pratilipi_id', 'read_percent',"updated_at"]]
interactions = pd.concat([interactions_current,interactions_new])

In [7]:
interactions_new.shape,interactions_current.shape

((61435, 4), (200799, 4))

In [8]:
# Since new users were removed at the time of creating data we don't have new user in test dataset
new_users = np.setdiff1d(interactions_new["user_id"], interactions_current["user_id"])
old_users = np.intersect1d(interactions_new["user_id"], interactions_current["user_id"])

print("Number of new users in new set: {}".format(len(new_users)))
print("Number of current users in new set: {}".format(len(old_users)))

Number of new users in new set: 0
Number of current users in new set: 11309


In [9]:
# Since new items(pratilipi) were removed at the time of creating data we don't have new user is test dataset

new_items = np.setdiff1d(interactions_new['pratilipi_id'], interactions_current['pratilipi_id'])
old_items = np.intersect1d(interactions_new['pratilipi_id'], interactions_current['pratilipi_id'])

print("Number of new items in new set: {}".format(len(new_items)))
print("Number of current items in new set: {}".format(len(old_items)))

Number of new items in new set: 0
Number of current items in new set: 24522


In [10]:
final_train.drop(["train"],inplace = True,axis = 1)
final_test.drop(["train"],inplace = True,axis = 1)

In [11]:
final_train.columns

Index(['user_id', 'pratilipi_id', 'read_percent', 'updated_at', 'month',
       'year', 'reading_time_min', 'genre|action', 'genre|adventure',
       'genre|and', 'genre|awards', 'genre|challenge', 'genre|chat',
       'genre|children', 'genre|comedy', 'genre|crime', 'genre|cyber',
       'genre|detective', 'genre|drama', 'genre|entertainment',
       'genre|erotica', 'genre|experiences', 'genre|family', 'genre|fantasy',
       'genre|fiction', 'genre|health', 'genre|hindi', 'genre|horror',
       'genre|indiawale', 'genre|inspiring', 'genre|jeevan', 'genre|kalamkar',
       'genre|kids', 'genre|krishi', 'genre|lekhan', 'genre|life',
       'genre|marathon', 'genre|memories', 'genre|moral', 'genre|murder',
       'genre|mystery', 'genre|mythology', 'genre|novels', 'genre|parytan',
       'genre|politics', 'genre|pratilipi', 'genre|pravasi',
       'genre|premkamahina', 'genre|radio', 'genre|rashtriya',
       'genre|relegion', 'genre|romance', 'genre|sahitya', 'genre|samman',
       'g

In [12]:
# List of item features we have for a single pratilipi
item_cols = ['user_id', 'pratilipi_id', 'read_percent', 'updated_at', 'month',
       'year', 'reading_time_min', 'genre|action', 'genre|adventure',
       'genre|and', 'genre|awards', 'genre|challenge', 'genre|chat',
       'genre|children', 'genre|comedy', 'genre|crime', 'genre|cyber',
       'genre|detective', 'genre|drama', 'genre|entertainment',
       'genre|erotica', 'genre|experiences', 'genre|family', 'genre|fantasy',
       'genre|fiction', 'genre|health', 'genre|hindi', 'genre|horror',
       'genre|indiawale', 'genre|inspiring', 'genre|jeevan', 'genre|kalamkar',
       'genre|kids', 'genre|krishi', 'genre|lekhan', 'genre|life',
       'genre|marathon', 'genre|memories', 'genre|moral', 'genre|murder',
       'genre|mystery', 'genre|mythology', 'genre|novels', 'genre|parytan',
       'genre|politics', 'genre|pratilipi', 'genre|pravasi',
       'genre|premkamahina', 'genre|radio', 'genre|rashtriya',
       'genre|relegion', 'genre|romance', 'genre|sahitya', 'genre|samman',
       'genre|science', 'genre|serieswriting', 'genre|short',
       'genre|shortstories', 'genre|social', 'genre|spiritual', 'genre|story',
       'genre|suspense', 'genre|swahindi2', 'genre|the', 'genre|translation',
       'genre|webseries', 'genre|wellness', 'genre|women']

In [13]:
item_features_current = final_train[item_cols].drop_duplicates(subset = ['pratilipi_id']).reset_index(drop = True)
item_features_new = final_test[item_cols].drop_duplicates(subset = ['pratilipi_id']).reset_index(drop = True)

In [14]:
item_features_current = item_features_current[item_features_current["pratilipi_id"].isin(interactions_current["pratilipi_id"])]
item_features_new = item_features_new[item_features_new["pratilipi_id"].isin(interactions_new["pratilipi_id"])]

print("Number of current items: {}".format(len(item_features_current)))
print("Number of new items: {}".format(len(item_features_new)))

Number of current items: 59407
Number of new items: 24522


In [15]:
item_features = pd.concat([item_features_current])

### Transform to LightFM format

#### Dataset
To train the LightFM model the dataset must be converted in a specific format.

In [16]:
print("Max user id: {}".format(interactions_current['user_id'].max()))
print("Max pratilipi id: {}".format(interactions_current['pratilipi_id'].max()))

Max user id: 5506791996684691
Max pratilipi id: 1377786228364236


In [17]:
user_ids_buffered = (x for x in interactions_current['user_id'].unique())
item_ids_buffered = (x for x in interactions_current['pratilipi_id'].unique())

item_feature_names = item_features.columns.tolist()
item_feature_names.remove("pratilipi_id")

In [18]:
dataset = Dataset()

In [19]:
dataset.fit(
    users=user_ids_buffered,
    items=item_ids_buffered,
    item_features=item_feature_names
)

In [20]:
def transform_interactions(interactions):
    """
    Transforms interactions data frame to the format build_interactions needs it.
    
    Returns iterable of tuples ("user_id", "pratilipi_id", 'read_percent').
    """
    return interactions[["user_id", "pratilipi_id", 'read_percent']].itertuples(index=False)

In [21]:
interaction_matrix_current, _ = dataset.build_interactions(transform_interactions(interactions_current))

In [22]:
interactions_current.shape

(200799, 4)

In [23]:
interaction_matrix_new, _ = dataset.build_interactions(transform_interactions(interactions_new))

In [24]:
interactions_new.shape

(61435, 4)

In [25]:
def transform_features(features, id_name):
    """
    Transforms features data frame to the format build_user_features / build_item_features needs it.
    
    Returns iterable of (id, [list of feature names that has value 1]).
    """
    transformed_features = []
    for row in features.to_dict(orient="records"):
        id_value = row[id_name]
        del row[id_name]
        feature_names = [key for key, value in row.items() if value == 1]
        transformed_features.append(
            (id_value, feature_names)
        )
    return transformed_features

In [26]:
item_features_matrix_current = dataset.build_item_features(transform_features(item_features_current, "pratilipi_id"))

In [27]:
item_features_matrix_new = dataset.build_item_features(transform_features(item_features_new, "pratilipi_id"))

## Training Model

In [28]:
model = LightFM(loss="warp",no_components=150, learning_rate=0.05,random_state=2019)

In [29]:
class Evaluator():
    """
    Helper class for model evaluation
    
    Stores the number of epochs and corresponding performance metrics in a data frame.
    Returns the number of epochs with the best validation ROC AUC.
    Prepared to handle multiple runs of the same hyperparameters, and averages up the multiple runs.
    """
    
    def __init__(self):
        self.df_evaluation = pd.DataFrame()
    def evaluate(self, epochs, model, interaction_matrix_train,item_features_matrix):
            features_current = {
               
                "item_features": item_features_matrix
            }
            evaluation_row = pd.DataFrame({
                "epochs": epochs,
                "train_precision": precision_at_k(model, interaction_matrix_train, k=10, **features_current).mean(),
                "train_auc": auc_score(model, interaction_matrix_train, **features_current).mean(),
                },
                index=[0]
            )
            self.df_evaluation = self.df_evaluation.append(evaluation_row, ignore_index=True)
    def get_best_epochs(self):
        df_evaluation_agg = self.df_evaluation.groupby("epochs").agg("mean").reset_index()
        best_epochs =  df_evaluation_agg[
            df_evaluation_agg["train_auc"] == df_evaluation_agg["train_auc"].max()
        ]["epochs"].iloc[0]
        return best_epochs

In [30]:
evaluator = Evaluator()
for epochs in [200]:
    print(f"Epochs: {epochs}")
    
    model.fit(
        interaction_matrix_current,
        epochs=epochs,
        item_features=item_features_matrix_current
    )
    
    evaluator.evaluate(
        epochs, model,
        interaction_matrix_current,item_features_matrix_current)

Epochs: 200


In [31]:
# train evealuation
evaluator.df_evaluation

,epochs,train_precision,train_auc
0,200,0.856872,0.999988


In [32]:
# test evaluation
features_new = {
               
                "item_features": item_features_matrix_new
            }

In [33]:
#AUC Score
print(auc_score(model, interaction_matrix_new, **features_new).mean())

0.87815267


In [34]:
#Precision at K
print(precision_at_k(model, interaction_matrix_new, k=10, **features_new).mean())

0.00864798


In [35]:
#Reciprocal Rank
print(reciprocal_rank(model, interaction_matrix_new, **features_new).mean())

0.03432402


In [36]:
#Recall at
print(recall_at_k(model, interaction_matrix_new, k=10, **features_new).mean())

0.026863608127034446


### Get recommendation for test dataset

In [37]:
def predict(model, dataset, user_item_pairs,item_features_matrix):
    user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = dataset.mapping()
    predictions = model.predict(
        user_ids=np.array([user_id_mapping[x] for x in user_item_pairs["user_id"]]),
        item_ids=np.array([item_id_mapping[x] for x in user_item_pairs["pratilipi_id"]]),
        item_features=item_features_matrix
    )
    
    df_predictions = user_item_pairs.copy()
    df_predictions["prediction"] = predictions
    
    return df_predictions

def merge_interactions(df_evaluation, interactions):
    df_evaluation = df_evaluation.merge(interactions[["user_id", "pratilipi_id", "read_percent"]], on=["user_id", "pratilipi_id"], how="left")
    df_evaluation["has_interaction"] = ~df_evaluation["read_percent"].isna()
#     df_evaluation = df_evaluation.drop(columns="read_percent")
    return df_evaluation

In [38]:
metadata = pd.read_csv("metadata.csv")

In [39]:
complete_test = final_test.merge(metadata,on = "pratilipi_id")

In [40]:
complete_test.head()

,user_id,pratilipi_id,read_percent,updated_at_x,month,year,reading_time_min,genre|action,genre|adventure,genre|and,...,genre|the,genre|translation,genre|webseries,genre|wellness,genre|women,author_id,category_name,reading_time,updated_at_y,published_at
0,5506791992372558,1377786218111595,100.0,2022-03-23 00:08:25.250,2.0,2020.0,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-2270332350793996,novels,1772,2020-11-23 08:28:17,2020-02-27 10:51:31
1,5506791992372558,1377786218111595,100.0,2022-03-23 00:08:25.250,2.0,2020.0,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-2270332350793996,romance,1772,2020-11-23 08:28:17,2020-02-27 10:51:31
2,5506791992372558,1377786218111595,100.0,2022-03-23 00:08:25.250,2.0,2020.0,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-2270332350793996,suspense,1772,2020-11-23 08:28:17,2020-02-27 10:51:31
3,5506791982337812,1377786218111595,100.0,2022-03-22 21:07:09.496,2.0,2020.0,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-2270332350793996,novels,1772,2020-11-23 08:28:17,2020-02-27 10:51:31
4,5506791982337812,1377786218111595,100.0,2022-03-22 21:07:09.496,2.0,2020.0,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-2270332350793996,romance,1772,2020-11-23 08:28:17,2020-02-27 10:51:31


In [41]:
g = {'pratilipi_id':lambda x: list(x)[:100] if len(list(x))>100 else list(x),
    "category_name":lambda x: list(x)}
final_test = complete_test.sort_values(['updated_at_x']).groupby(["user_id"]).agg(g).reset_index()

In [42]:
final_test["predicted_id"] = None

In [43]:
final_test.head()

,user_id,pratilipi_id,category_name,predicted_id
0,3263998672675492,"[1377786225588601, 1377786225687358, 137778622...","[shortstories, shortstories, shortstories, sus...",None
1,3264359729373860,"[1377786217840312, 1377786217840312, 137778621...","[suspense, novels, romance, novels, romance, s...",None
2,3264802291360420,"[1377786223781436, 1377786223781436, 137778622...","[horror, suspense, entertainment, entertainmen...",None
3,3308162227544740,"[1377786227359359, 1377786227359359, 137778622...","[Serieswriting, romance, social]",None
4,3314890925580964,"[1377786222333773, 1377786222333773, 137778622...","[family, romance, suspense, suspense, romance,...",None


In [44]:
all_item_ids = np.unique(interactions["pratilipi_id"])

for i in range(1000):
#     try:
    user_sample_item_pairs = pd.DataFrame({
        "user_id": final_test["user_id"][i],
        "pratilipi_id": all_item_ids
    })
    predictions_sample = predict(model, dataset, user_sample_item_pairs,item_features_matrix_new )
    predictions_sample = merge_interactions(predictions_sample, interactions)
    predictions_sample = predictions_sample.sort_values("prediction", ascending=False)
    final_test["predicted_id"][i] = list(set(predictions_sample["pratilipi_id"].tolist()[:100]))

C:\Users\sprat\AppData\Local\Temp\ipykernel_9792\3053470904.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test["predicted_id"][i] = list(set(predictions_sample["pratilipi_id"].tolist()[:100]))


In [45]:
final_test=final_test[:1000]

In [46]:
### For better understanding of the recommendation we will also find category of predicted items
g = {"category_name":lambda x: list(x)}
metadict = metadata.groupby(["pratilipi_id"]).agg(g).reset_index()

In [47]:
metadata.shape

(954501, 6)

In [48]:
metadict["pratilipi_id"] = metadict.apply(lambda x: abs(x["pratilipi_id"]),axis = 1)

In [49]:
dic = {}
for i in range(len(metadict)):
    dic[metadict["pratilipi_id"][i]] = metadict["category_name"][i]

In [50]:
final_test["predicted_cat"] = None

In [51]:
for i in range(len(final_test)):
    new_lis = []
    for j in range(len(final_test["predicted_id"][i])):
        new_lis = new_lis + dic[final_test["predicted_id"][i][j]]
    final_test["predicted_cat"][i] = list(set(new_lis))

C:\Users\sprat\AppData\Local\Temp\ipykernel_9792\179527071.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test["predicted_cat"][i] = list(set(new_lis))


In [52]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

In [53]:
final_test.head()

,user_id,pratilipi_id,category_name,predicted_id,predicted_cat
0,3263998672675492,"[1377786225588601, 1377786225687358, 137778622...","[shortstories, shortstories, shortstories, sus...","[1377786226439685, 1377786226153480, 137778622...","[horror, moral-inspiring, romance, action-and-..."
1,3264359729373860,"[1377786217840312, 1377786217840312, 137778621...","[suspense, novels, romance, novels, romance, s...","[1377786219134467, 1377786216199683, 137778622...","[action-and-adventure, webseries, horror, soci..."
2,3264802291360420,"[1377786223781436, 1377786223781436, 137778622...","[horror, suspense, entertainment, entertainmen...","[1377786216201740, 1377786222195733, 137778621...","[action-and-adventure, life, science-fiction, ..."
3,3308162227544740,"[1377786227359359, 1377786227359359, 137778622...","[Serieswriting, romance, social]","[1377786226714126, 1377786220894223, 137778621...","[action-and-adventure, Pratilipi-kalamkar-samm..."
4,3314890925580964,"[1377786222333773, 1377786222333773, 137778622...","[family, romance, suspense, suspense, romance,...","[1377786221899267, 1377786228003332, 137778622...","[action-and-adventure, life, webseries, horror..."


In [54]:
final_test["jac_score"] = final_test.apply(lambda x : jaccard_similarity(x["pratilipi_id"],x["predicted_id"]),axis = 1)

In [55]:
final_test["jac_score"].mean()

0.02066706369836303

In [56]:
final_test["cat_jac_score"] = final_test.apply(lambda x : jaccard_similarity(x["category_name"],x["predicted_cat"]),axis = 1)

In [57]:
final_test["cat_jac_score"].mean()

0.16669303166699856

In [58]:
final_test.head()

,user_id,pratilipi_id,category_name,predicted_id,predicted_cat,jac_score,cat_jac_score
0,3263998672675492,"[1377786225588601, 1377786225687358, 137778622...","[shortstories, shortstories, shortstories, sus...","[1377786226439685, 1377786226153480, 137778622...","[horror, moral-inspiring, romance, action-and-...",0.018519,0.294118
1,3264359729373860,"[1377786217840312, 1377786217840312, 137778621...","[suspense, novels, romance, novels, romance, s...","[1377786219134467, 1377786216199683, 137778622...","[action-and-adventure, webseries, horror, soci...",0.027273,0.150000
2,3264802291360420,"[1377786223781436, 1377786223781436, 137778622...","[horror, suspense, entertainment, entertainmen...","[1377786216201740, 1377786222195733, 137778621...","[action-and-adventure, life, science-fiction, ...",0.000000,0.103448
3,3308162227544740,"[1377786227359359, 1377786227359359, 137778622...","[Serieswriting, romance, social]","[1377786226714126, 1377786220894223, 137778621...","[action-and-adventure, Pratilipi-kalamkar-samm...",0.000000,0.064516
4,3314890925580964,"[1377786222333773, 1377786222333773, 137778622...","[family, romance, suspense, suspense, romance,...","[1377786221899267, 1377786228003332, 137778622...","[action-and-adventure, life, webseries, horror...",0.000000,0.157895


In [59]:
final_test.to_csv("Recommendation.csv",index = False)